<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/distilbert_cased_german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 8.1MB/s 
     |████████████████████████████████| 3.2MB 35.9MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=db2873feed951fcaf68a4831abcea6ac557a711bb63f361c59a429e1743c1481
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:38:40--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:38:40 (53.0 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-german-cased')
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-german-cased')

sequence = 'Ich wohne in {}, und ich spreche {}.'

rows = []
for name in pd.read_csv('countries.csv')['de']:
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

In [4]:
pd.DataFrame(rows, columns=['Land', 'Sprache(top1)', 'Sprache(top2)', 'Sprache(top3)'])

,Land,Sprache(top1),Sprache(top2),Sprache(top3)
0,Afghanistan,Englisch (27%),Deutsch (24%),Französisch (9%)
1,Albanien,Deutsch (32%),Englisch (14%),deutsch (13%)
2,Algerien,Französisch (38%),französisch (16%),Deutsch (12%)
3,Andorra,Spanisch (47%),Französisch (25%),Englisch (7%)
4,Angola,Französisch (31%),Englisch (18%),Spanisch (11%)
5,Antigua und Barbuda,Spanisch (30%),Französisch (27%),Englisch (18%)
6,Argentinien,Spanisch (48%),Französisch (12%),Englisch (9%)
7,Armenien,Englisch (19%),Deutsch (18%),Französisch (14%)
8,Australien,Englisch (47%),Deutsch (9%),englisch (9%)
9,Österreich,Deutsch (49%),deutsch (13%),Englisch (13%)
